_05839: Interactive Data Science <br> Instructor: John Stamper_

_Final Project_

__Topic: Plastic Pollution__<br>
__Submitted by: mmalani__

_Data and information reference: https://ourworldindata.org/plastic-pollution_

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json, ast
import plotly.express as px
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

sns.set()
%matplotlib inline

import chardet
sns.set(style="whitegrid", color_codes=True)

In [14]:
! pip install plotly

In [35]:
cumulative_df = pd.read_csv('PlasticPollution_Datasets_Edited\cumulative-global-plastics_Edited.csv')

In [36]:
# Prediction Model - Linear/RBF/Polynomial Regression

def prediction_model(data, ml_type, col):
    
    # Source - https://scikit-learn.org/stable/auto_examples/svm/plot_svm_regression.html
 
    year_list = data["Year"].tolist()
    year_list.sort()
    years = np.array(data["Year"].tolist()).reshape(-1,1)
    values = np.array(data[col].tolist()).reshape(-1,1)

    # Train
    svr_type = None
    # print(type)
    if ml_type == "RBF":
        svr_type= SVR(kernel="rbf", C=1, gamma=0.1, epsilon=.1)
    elif ml_type == "Linear":
        svr_type= SVR(kernel="linear", C=1, gamma='auto')
    elif ml_type == "Polynomial":
        svr_type= SVR(kernel="poly", C=1, gamma='auto', degree=3, epsilon=.1)

    sc_X = StandardScaler()
    sc_y = StandardScaler()
    X = sc_X.fit_transform(years)
    y = sc_y.fit_transform(values)

    svr = svr_type.fit(X,y)

    # Generate predicted values
    predYears = []
    futureVals = []
    
    for i in range(year_list[0], year_list[-1]+1):
        predYears.append(i)
        futureVals.append(data[data["Year"] == i][col].tolist()[0])
    
    for i in range(year_list[-1]+1,2036):
        predYears.append(i)
        y_pred = sc_y.inverse_transform((svr.predict(sc_X.transform(np.array([[i]]))).reshape(1, -1) ))
        #temp_df = temp_df.append({"Entity": c, "Code": data["Code"].values[0], 
        #                        "Year": i, "Number of internet users (OWID based on WB & UN)" : y_pred[0][0]}, 
        #                        ignore_index = True)
        futureVals.append(y_pred[0][0])

    temp_df = pd.DataFrame({
            "Year": predYears,
            col + "_" + ml_type: futureVals 
    })
    return temp_df

In [37]:
cumulative_df.head()

,Unnamed: 0,Year,Production (million tonnes)
0,0,1950,2000000
1,1,1951,2000000
2,2,1952,2000000
3,3,1953,3000000
4,4,1954,3000000


In [38]:
col = 'Production (million tonnes)'
cum_RBF = prediction_model(cumulative_df, "RBF", col)
cum_Linear = prediction_model(cumulative_df, "Linear", col)
cum_Polynomial = prediction_model(cumulative_df, "Polynomial", col)

C:\Users\monal\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\monal\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\monal\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [39]:
cum_RBF.head()

,Year,Production (million tonnes)_RBF
0,1950,2000000.0
1,1951,2000000.0
2,1952,2000000.0
3,1953,3000000.0
4,1954,3000000.0


In [40]:
cum_Linear.tail()

,Year,Production (million tonnes)_Linear
81,2031,3.732420e+08
82,2032,3.786047e+08
83,2033,3.839674e+08
84,2034,3.893300e+08
85,2035,3.946927e+08


In [41]:
cum_Polynomial.tail()

,Year,Production (million tonnes)_Polynomial
81,2031,1.058302e+09
82,2032,1.117898e+09
83,2033,1.179952e+09
84,2034,1.244512e+09
85,2035,1.311629e+09


In [45]:
df = cumulative_df.copy()
df = df.join(cum_RBF.set_index('Year'), on='Year', how="right")
df = df.join(cum_Linear.set_index('Year'), on='Year', how="right")
df = df.join(cum_Polynomial.set_index('Year'), on='Year', how='right')
df

,Unnamed: 0,Year,Production (million tonnes),Production (million tonnes)_RBF,Production (million tonnes)_Linear,Production (million tonnes)_Polynomial
0.0,0.0,1950,2000000.0,2.000000e+06,2.000000e+06,2.000000e+06
1.0,1.0,1951,2000000.0,2.000000e+06,2.000000e+06,2.000000e+06
2.0,2.0,1952,2000000.0,2.000000e+06,2.000000e+06,2.000000e+06
3.0,3.0,1953,3000000.0,3.000000e+06,3.000000e+06,3.000000e+06
4.0,4.0,1954,3000000.0,3.000000e+06,3.000000e+06,3.000000e+06
...,...,...,...,...,...,...
NaN,NaN,2031,NaN,4.238937e+08,3.732420e+08,1.058302e+09
NaN,NaN,2032,NaN,4.273256e+08,3.786047e+08,1.117898e+09
NaN,NaN,2033,NaN,4.304907e+08,3.839674e+08,1.179952e+09
NaN,NaN,2034,NaN,4.333880e+08,3.893300e+08,1.244512e+09


In [46]:
df.to_csv("PlasticPollution_Datasets_Edited\cumulative-global-plastics_Edited_ML.csv")

In [47]:
global_df = pd.read_csv('PlasticPollution_Datasets_Edited\global-plastics-production_Edited.csv')
global_df.head()

,Unnamed: 0,Year,Production (million tonnes)
0,0,1950,2000000
1,1,1951,2000000
2,2,1952,2000000
3,3,1953,3000000
4,4,1954,3000000


In [48]:
col = 'Production (million tonnes)'
global_df_RBF = prediction_model(global_df, "RBF", col)
global_df_Linear = prediction_model(global_df, "Linear", col)
global_df_Polynomial = prediction_model(global_df, "Polynomial", col)
df = global_df.copy()
df = df.join(global_df_RBF.set_index('Year'), on='Year', how="right")
df = df.join(global_df_Linear.set_index('Year'), on='Year', how="right")
df = df.join(global_df_Polynomial.set_index('Year'), on='Year', how='right')

C:\Users\monal\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\monal\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\monal\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [50]:
df

,Unnamed: 0,Year,Production (million tonnes),Production (million tonnes)_RBF,Production (million tonnes)_Linear,Production (million tonnes)_Polynomial
0.0,0.0,1950,2000000.0,2.000000e+06,2.000000e+06,2.000000e+06
1.0,1.0,1951,2000000.0,2.000000e+06,2.000000e+06,2.000000e+06
2.0,2.0,1952,2000000.0,2.000000e+06,2.000000e+06,2.000000e+06
3.0,3.0,1953,3000000.0,3.000000e+06,3.000000e+06,3.000000e+06
4.0,4.0,1954,3000000.0,3.000000e+06,3.000000e+06,3.000000e+06
...,...,...,...,...,...,...
NaN,NaN,2031,NaN,4.238937e+08,3.732420e+08,1.058302e+09
NaN,NaN,2032,NaN,4.273256e+08,3.786047e+08,1.117898e+09
NaN,NaN,2033,NaN,4.304907e+08,3.839674e+08,1.179952e+09
NaN,NaN,2034,NaN,4.333880e+08,3.893300e+08,1.244512e+09


In [51]:
df.to_csv("PlasticPollution_Datasets_Edited\global-plastics-production_Edited_ML.csv")